# Logistic Regression

In [1]:
# 依赖，使用torch构建模型
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from utils.utils import CriteoDataset, CTRMetric

# 1. 数据集处理

In [2]:
# 使用的超参数
config = {
    'TRAIN_BATCH_SIZE': 128,
    'VALID_BATCH_SIZE': 128,
    'TEST_BATCH_SIZE': 128,
    'DEVICE': 'mps',
    'NUM_WORKERS': 6,
    'EPOCH': 30,
    'NUM_FEATURE': 111,
    'POS_WEIGHT': 4,
    'LEARNING_RATE': 1e-4,
}


In [3]:
# 使用的数据集为Criteo数据集
train_pth = '../dataset/criteo-100k-train.txt'
valid_pth = '../dataset/criteo-100k-valid.txt'
test_pth = '../dataset/criteo-100k-test.txt'
train_set = CriteoDataset(train_pth, mode='train')
valid_set = CriteoDataset(valid_pth, mode='valid', encoders=train_set.encoders)
test_set = CriteoDataset(valid_pth, mode='test', encoders=train_set.encoders)

/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [4]:
train_loader = DataLoader(
    dataset=train_set,
    batch_size=config['TRAIN_BATCH_SIZE'],
    shuffle=True,
    num_workers=config['NUM_WORKERS']
)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=config['VALID_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=config['TEST_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

# for e in range(2):
#     for step, (batch_y, batch_X) in enumerate(tqdm(train_loader)):
#         print(f'epoch: {e}\tstep: {step}\tbatch_X: {batch_X}\tbatch_y: {batch_y}')
#         if step >= 10:
#             break

In [5]:
# 构建模型
class LogisticRegressionModel(nn.Module):
    def __init__(self, num_features):
        super(LogisticRegressionModel, self).__init__()
        self.bn = nn.BatchNorm1d(num_features=num_features)
        self.linar_layer = nn.Linear(num_features, 1)

    def forward(self, batch_X):
        return self.linar_layer(self.bn(batch_X))

In [6]:
# 构建trianer
class Trainer:
    def __init__(self, model, loss_func, optimizer, metric, train_loader, valid_loader, test_loader, config):
        self.config = config
        print('='*10 + "Config" + '='*10)
        for k, v in self.config.items():
            print(f'{k}: {v}')
        print('='*25)
        self.model: nn.Module = model
        self.loss_func: nn.Module = loss_func
        self.optimizer: optim.Optimizer = optimizer
        self.metric = metric
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader

        self.to(self.config['DEVICE'])

    def to(self, device=None):
        if device is None:
            self.model = self.model.to(self.config['DEVICE'])
            self.loss_func = self.loss_func.to(self.config['DEVICE'])
        else:
            self.model = self.model.to(device)
            self.loss_func = self.loss_func.to(self.config['DEVICE'])
            self.config['DEVICE'] = device

    def step(self, batch_y, batch_x, mode='train'):
        device = self.config['DEVICE']
        if mode == 'train':
            self.model.train()
            self.optimizer.zero_grad()

            batch_y = batch_y.to(device)
            batch_x = batch_x.to(device)

            pred = self.model(batch_x)
            loss = self.loss_func(pred, batch_y)
            loss.backward()
            self.optimizer.step()
            return loss.item(), pred
        elif mode == 'evaluate':
            with torch.no_grad():
                self.model.eval()

                batch_y = batch_y.to(device)
                batch_x = batch_x.to(device)

                pred = self.model(batch_x)
                loss = self.loss_func(pred, batch_y)
                return loss.item(), pred
        else:
            raise ValueError("Wrong Mode")
    def _compute_metric(self, metric_str):
        self.metric.get_batch_metric()
        for k, v in self.metric.metric_dict.items():
            if k == 'cnt':
                continue
            metric_str += f'{k}: {v:4f}\n'
        self.metric.init_metric()
        return metric_str
    def train(self):
        print("="*10 + "TRAIN BEGIN" + "="*10)
        epoch = self.config['EPOCH']
        for e in range(1, epoch + 1):
            all_loss = 0.0
            for s, (batch_y, batch_x) in enumerate(tqdm(train_loader)):
                loss, pred = self.step(batch_y.unsqueeze(1), batch_x, mode='train')
                all_loss += loss

            all_loss /= s + 1
            print(f'Train Epoch: {e}\nLoss: {all_loss}')
            if e % 1 == 0:
                all_loss = 0.0
                self.metric.init_metric()
                for s, (batch_y, batch_x) in enumerate(tqdm(valid_loader)):
                    loss, pred = self.step(batch_y.unsqueeze(1), batch_x, mode='evaluate')
                    all_loss += loss

                    self.metric.compute_metric(pred, batch_y)
                all_loss /= s + 1
                metric_str = f'loss: {all_loss}\n'
                metric_str = self._compute_metric(metric_str)
                print(f'Valid Epoch: {e}\n' + metric_str)
        print("="*10 + "TRAIN END" + "="*10)

    def test(self):
        all_loss = 0.0
        self.metric.init_metric()
        for s, (batch_y, batch_x) in enumerate(tqdm(test_loader)):
            loss, pred = self.step(batch_y.unsqueeze(1), batch_x, mode='evaluate')
            all_loss += loss

        all_loss /= s + 1
        metric_str = f'loss: {all_loss}\n'
        metric_str = self._compute_metric(metric_str)
        print(f'Test Loss: {all_loss}\n' + metric_str)

In [7]:
model = LogisticRegressionModel(num_features=config['NUM_FEATURE'])
weight = torch.tensor(config['POS_WEIGHT'])
loss_func = nn.BCEWithLogitsLoss(pos_weight=weight)
optimizer = optim.Adam(lr=config['LEARNING_RATE'], params=model.parameters())
metric = CTRMetric()

trainer = Trainer(
    model=model,
    loss_func=loss_func,
    optimizer=optimizer,
    metric=metric,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    config=config
)

==========Config==========
TRAIN_BATCH_SIZE: 128
VALID_BATCH_SIZE: 128
TEST_BATCH_SIZE: 128
DEVICE: mps
NUM_WORKERS: 6
EPOCH: 30
NUM_FEATURE: 111
POS_WEIGHT: [4]
LEARNING_RATE: 0.0001


In [8]:
if __name__ == '__main__':
    trainer.train()
    trainer.test()

==========TRAIN BEGIN==========


100%|██████████| 625/625 [00:09<00:00, 67.62it/s] 


Train Epoch: 1
Loss: 1.0577327836036683


100%|██████████| 79/79 [00:07<00:00, 11.21it/s]


Valid Epoch: 1
loss: 1.0555117107644867
accuracy: 0.737342
precision: 0.429355
recall: 0.428049
F1: 0.424967
AUC: 0.710751



...

Valid Epoch: 22
loss: 1.0627609619611427
accuracy: 0.750297
precision: 0.455396
recall: 0.492948
F1: 0.470192
AUC: 0.744255



100%|██████████| 625/625 [00:08<00:00, 70.72it/s] 


Train Epoch: 23
Loss: 0.7746822437286377


  0%|          | 0/79 [00:03<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/xansar/PycharmProjects/RecommenderSystem/Recommender-System-Pytorch/MyImplement/utils/utils.py", line 3, in <module>
    import pandas_profiling as pp
  File "/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/site-packages/pandas_profiling/__init__.py", line 6, in <module>
    from pandas_profiling.controller import pandas_decorator
  File "/Users/xansar/.conda/envs/RecommenderSystem/lib/python3.8/site-packages/pandas_profiling/controller/pandas_decorator.py", line 4, in <module>
    from pandas_profiling.profile_report import ProfileReport
 

KeyboardInterrupt: 